# Overview of [smolagents](https://huggingface.co/docs/smolagents)

In [ ]:
import os

from smolagents import LiteLLMModel, ToolCallingAgent

from travel_agent.qdrant import client as qdrant_client
from travel_agent.retrieval.embedding.embedding_generation import MODELS_PROMPTS
from travel_agent.retrieval.smolagents.prompts import RUS_PROMPT_TEMPLATES
from travel_agent.retrieval.smolagents.tool import TravelReviewQueryTool

if not os.getenv("QDRANT_URL"):
    print("Set QDRANT_URL and QDRANT_API_KEY env")
    exit(1)

print(f"Available embed models:\n- {'\n- '.join(MODELS_PROMPTS.keys())}")

2025-04-20 08:37:45.455 | DEBUG    | travel_agent.qdrant.client:create_client:12 - Connecting to Qdrant at https://21a1c8c4-0d63-4176-9930-ee7848dafd2f.eu-central-1-0.aws.cloud.qdrant.io
2025-04-20 08:37:46.212 | SUCCESS  | travel_agent.qdrant.client:create_client:22 - connected: title='qdrant - vector search engine' version='1.13.6' commit='4db98ecdbf66bedbce268ec26865ea446d0bba7b'


Available embed models:
- cointegrated/rubert-tiny2
- DeepPavlov/rubert-base-cased-sentence
- ai-forever/sbert_large_nlu_ru
- ai-forever/sbert_large_mt_nlu_ru
- sentence-transformers/distiluse-base-multilingual-cased-v1
- sentence-transformers/distiluse-base-multilingual-cased-v2
- sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
- sentence-transformers/paraphrase-multilingual-mpnet-base-v2
- intfloat/multilingual-e5-large
- intfloat/multilingual-e5-base
- intfloat/multilingual-e5-small
- ai-forever/ru-en-RoSBERTa
- sergeyzh/BERTA


In [2]:
# Replace with your favourite local model
MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0"
model_id = f"ollama_chat/{MODEL_NAME}"


model = LiteLLMModel(
    model_id=model_id,
    api_base=os.getenv("OLLAMA_URL", "http://127.0.0.1:11434"),
    num_ctx=8192,  # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
)

test_response = model([{"role": "user", "content": [{"text": "Алло, это Кизару у телефона?", "type": "text"}]}]).content
print(test_response)
assert test_response

Привет! Да, это я, Кизару. - *улыбается* Как я могу помочь вам сегодня?


In [3]:
embed_model_name = "intfloat/multilingual-e5-base"
collection_name = "moskva_intfloat_multilingual_e5_base"


review_search_tool = TravelReviewQueryTool(embed_model_name, qdrant_client, collection_name)


agent = ToolCallingAgent(tools=[review_search_tool], model=model, max_steps=3, verbosity_level=2)

2025-04-20 08:38:05.644 | INFO     | travel_agent.retrieval.smolagents.tool:__init__:38 - Using device: mps
No sentence-transformers model found with name intfloat/multilingual-e5-base. Creating a new one with mean pooling.


In [4]:
print(agent.system_prompt)

You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer"

In [5]:
# Sanity check
agent.run("Что ты думаешь о римской империи?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Что ты думаешь о римской империи?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
"name": "final_answer",                                                                                            
"arguments": {"answer": "Римская империя была одним из величайших государственных образований в истории            
человечества. Вот краткий обзор её основных аспектов:\n- Просуществовала более тысячи лет (примерно с 27 до н.э. по
476 н.э.)\n- Объединила под своим управлением огромные территории, включая большую часть Европы и Ближнего         
Востока\n- Была центром искусства, архитектуры, права и литературы\n- Развивала сложную административную систему и 
правовую основу (право Рима оказало значительное влияние на современное право)\n- Внесла вклад в развитие          
инфраструктуры через строительство дорог и водопроводных систем\n- Прошла через периоды расцвета и кризиса,        
завершив свое существование разделением на Западную и Восточную Римскую империи."}                                 
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Римская империя была одним из величайших               │
│ государственных образований в истории человечества. Вот краткий обзор её основных аспектов:\n- Просуществовала  │
│ более тысячи лет (примерно с 27 до н.э. по 476 н.э.)\n- Объединила под своим управлением огромные территории,   │
│ включая большую часть Европы и Ближнего Востока\n- Была центром искусства, архитектуры, права и литературы\n-   │
│ Развивала сложную административную систему и правовую основу (право Рима оказало значительное влияние на        │
│ современное право)\n- Внесла вклад в развитие инфраструктуры через строительство дорог и водопроводных          │
│ систем\n- Прошла через периоды расцвета и кризиса, завершив свое существование разделением на Западную и        │
│ Восточную Римскую империи.'}                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Римская империя была одним из величайших государственных образований в истории человечества. Вот 
краткий обзор её основных аспектов:
- Просуществовала более тысячи лет (примерно с 27 до н.э. по 476 н.э.)
- Объединила под своим управлением огромные территории, включая большую часть Европы и Ближнего Востока
- Была центром искусства, архитектуры, права и литературы
- Развивала сложную административную систему и правовую основу (право Рима оказало значительное влияние на 
современное право)
- Внесла вклад в развитие инфраструктуры через строительство дорог и водопроводных систем
- Прошла через периоды расцвета и кризиса, завершив свое существование разделением на Западную и Восточную Римскую 
империи.

[Step 1: Duration 37.36 seconds| Input tokens: 1,252 | Output tokens: 207]

'Римская империя была одним из величайших государственных образований в истории человечества. Вот краткий обзор её основных аспектов:\n- Просуществовала более тысячи лет (примерно с 27 до н.э. по 476 н.э.)\n- Объединила под своим управлением огромные территории, включая большую часть Европы и Ближнего Востока\n- Была центром искусства, архитектуры, права и литературы\n- Развивала сложную административную систему и правовую основу (право Рима оказало значительное влияние на современное право)\n- Внесла вклад в развитие инфраструктуры через строительство дорог и водопроводных систем\n- Прошла через периоды расцвета и кризиса, завершив свое существование разделением на Западную и Восточную Римскую империи.'

In [6]:
resp = agent.run("Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?                                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
  "name": "travel_review_query",                                                                                   
  "arguments": {"query": "посоветуй хороший японский ресторан в Москве"}                                           
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'посоветуй хороший японский ресторан в Москве'}   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
    "name": "final_answer",                                                                                        
    "arguments": {                                                                                                 
        "answer": "Основываясь на полученных отзывах, рекомендую посетить ресторан 'Izumi' по адресу Мясницкая     
улица, д.38с1 в Москве. Отзывы отмечают разнообразие блюд, качественное обслуживание и приятную атмосферу. Также   
стоит обратить внимание на 'Tsunami Sushi & Cocktails', расположенный по адресу Благовещенский переулок, д.1А,     
особенно если вы цените не только еду, но и современную атмосферу."                                                
    }                                                                                                              
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "Основываясь на полученных отзывах, рекомендую посетить │
│ ресторан 'Izumi' по адресу Мясницкая улица, д.38с1 в Москве. Отзывы отмечают разнообразие блюд, качественное    │
│ обслуживание и приятную атмосферу. Также стоит обратить внимание на 'Tsunami Sushi & Cocktails', расположенный  │
│ по адресу Благовещенский переулок, д.1А, особенно если вы цените не только еду, но и современную атмосферу."}   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Основываясь на полученных отзывах, рекомендую посетить ресторан 'Izumi' по адресу Мясницкая улица, 
д.38с1 в Москве. Отзывы отмечают разнообразие блюд, качественное обслуживание и приятную атмосферу. Также стоит 
обратить внимание на 'Tsunami Sushi & Cocktails', расположенный по адресу Благовещенский переулок, д.1А, особенно 
если вы цените не только еду, но и современную атмосферу.

[Step 2: Duration 37.34 seconds| Input tokens: 3,964 | Output tokens: 185]

In [7]:
print(resp)

Основываясь на полученных отзывах, рекомендую посетить ресторан 'Izumi' по адресу Мясницкая улица, д.38с1 в Москве. Отзывы отмечают разнообразие блюд, качественное обслуживание и приятную атмосферу. Также стоит обратить внимание на 'Tsunami Sushi & Cocktails', расположенный по адресу Благовещенский переулок, д.1А, особенно если вы цените не только еду, но и современную атмосферу.


In [8]:
agent.memory.steps

[TaskStep(task='Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?', task_images=None),
 ActionStep(model_input_messages=[{'role': <MessageRole.SYSTEM: 'system'>, 'content': [{'type': 'text', 'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\

In [ ]:
# translated prompts

agent = ToolCallingAgent(
    tools=[review_search_tool], model=model, prompt_templates=RUS_PROMPT_TEMPLATES, max_steps=3, verbosity_level=2
)

print(agent.system_prompt)

Вы — эксперт-ассистент, который может решать любые задачи с использованием вызовов инструментов. Вам будет дана задача, которую вы должны решить наилучшим образом.
Для этого вам предоставлен доступ к некоторым инструментам.
Вызов инструмента, который вы пишете, является действием: после выполнения инструмента вы получите результат вызова инструмента как "наблюдение".
Эта последовательность Действие/Наблюдение может повторяться N раз, вам следует выполнять несколько шагов, когда это необходимо.
Вы можете использовать результат предыдущего действия как ввод для следующего действия.
Наблюдение всегда будет строкой: она может представлять файл, например "image_1.jpg".
Затем вы можете использовать его как ввод для следующего действия. Вы можете сделать это, например, следующим образом:

Наблюдение: "image_1.jpg"

Действие:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}
Чтобы предоставить окончательный ответ на задачу, используйте блок действия с "name": "final_an

In [11]:
resp = agent.run("Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?                                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{"name": "travel_review_query", "arguments": {"query": "Посоветуй хороший японский ресторан в Москве."}}           

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'Посоветуй хороший японский ресторан в Москве.'}  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Найденные отзывы о местах:

=== Отзыв на Izumi ===
Адрес: Москва, Мясницкая улица, 38с1
Рейтинг: 5.0
Категории: Ресторан;Бар, паб;Караоке-клуб
Текст: Одно из самых любимых заведений японской кухни в Москве!  За адекватную цену вы получите действительно 
вкусное, похожее на настоящее, японское блюдо. Разнообразное меню и изобилие различных видов саке. Приятный 
интерьер. Отдельно хочется отметить удобную функцию вызова официанта с помощью кнопочки на столе :) 

=== Отзыв на Tsunami Sushi & Cocktails ===
Адрес: Москва, Благовещенский переулок, 1А
Рейтинг: 5.0
Категории: Ресторан;Бар, паб;Суши-бар
Текст: Бомбически! Хотела поесть действительно вкусных роллов и не прогадала с выбором места. Приятный интерьер в 
стиле неонового Токио, техно/хаус музыка, которая не отвлекает от ужина, а только добавляет атмосферы. Посадка 
практически полная, но разговоры не мешали, шумно не было.  Брали роллы Мега Лосось, Радуга, Нигири с лососем и оши
с гребешком. Очень вкусно и очень сытно. Рыба свежая, сочетания вкусов потрясающие. Муж брал коктейли и тоже 
остался доволен.  UPD 17 июня 2023: Выбрала Tsunami, чтобы вдвоём с мужем отпраздновать мой день рождения. Как 
всегда безукаризненное обслуживание, наивкуснейшая еда, шикарная атмосфера. Пока люимое заведение японской кухни в 
Москве, куда действительно хочется возвращаться. 

=== Отзыв на Якитория ===
Адрес: Москва, улица Маршала Полубоярова, 1
Рейтинг: 5.0
Категории: Суши-бар;Ресторан;Доставка еды и обедов
Текст: Очень хороший  ресторанчик японской кухни. Заказываю всегда отсюда если хочется роллов. Очень вкусные 
горячие роллы. Детям также будет интересно перекусить тут. 

=== Отзыв на Izumi ===
Адрес: Москва, Мясницкая улица, 38с1
Рейтинг: 5.0
Категории: Ресторан;Бар, паб;Караоке-клуб
Текст: Прекрасное место с великолепным тихим антуражем. Был днём, с друзьями. Еда от японского шефа на любой вкус, 
и для тех кто не любит рыбу и морепродукты и для тех кто их обожает. Я лично ел Тонкацу Рамен, мне очень 
понравилось, в отличии от большинства мест в Москве где всëтаки готовится под ожидания русских, тут именно что 
необычный вкус явно не из того  тех что можно попробовать в други местах.   Именно тут я пожалуй понял что именно 
японцы имеют ввиду по умами вкусом, реально очень плотный и при этом совершенно нежирный бульон просто пропитан 
вкусом который в других кухнях встречается довольно редко даже при самом поедании мяса. Однозначно рекомендую, 
однако предупреждаю что напитки как алко так и без состоят изо льда на 80%. 

=== Отзыв на Каппо Хироки Аракава ===
Адрес: Москва, улица Охотный Ряд, 2
Рейтинг: 5.0
Категории: Ресторан;Суши-бар
Текст: С точки зрения продукта и еды, что ты получаешь это абсолютное качество. Мало того что все свежее, так ещё и
вкус каждого блюда просто отточен. Водоросли настоящие, а не то что обычно в японских ресторанах приносят. За 
маринованный имбирь, который приносят к суши просто душу можно отдать, моти нежнейшее от и до... Реально как в 
Токио сгонял, только все ближе ))) Обслуживание если вас будет обслуживать Эльвира будет высочайшим. Официант 
который был ещё что называется не але ))) В общем отличный настоящий японский ресторан, после которого почти всем 
японским модным ресторанам Москвы должно быть стыдно... Единственная некрасивая мелочь это заваленная бумагами и 
всякой канцелярщиной барная стойка. Но я так, докопаться Однозначно рекомендую. Я в следующий раз на омакасе сэт 
пойду...

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{ "name": "final_answer", "arguments": {"answer": "На основе отзывов рекомендуется посетить следующие японские     
рестораны в Москве:\n\n- **Izumi**\n• Адрес: Москва, Мясницкая улица, 38с1\n• Рейтинг: 5.0\n• Особенности: Одно из 
самых любимых мест с разнообразным меню и удобной системой вызова официантов.\n\n- **Tsunami Sushi & Cocktails**\n•
Адрес: Москва, Благовещенский переулок, 1А\n• Рейтинг: 5.0\n• Особенности: Отличное место с атмосферным интерьером 
и вкусными роллами.\n\n- **Якитория**\n• Адрес: Москва, улица Маршала Полубоярова, 1\n• Рейтинг: 5.0\n•            
Особенности: Хороший выбор для любителей горячих роллов и детских семейных посещений.\n\nВыбирайте на свой вкус!"} 
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'На основе отзывов рекомендуется посетить следующие     │
│ японские рестораны в Москве:\n\n- **Izumi**\n• Адрес: Москва, Мясницкая улица, 38с1\n• Рейтинг: 5.0\n•          │
│ Особенности: Одно из самых любимых мест с разнообразным меню и удобной системой вызова официантов.\n\n-         │
│ **Tsunami Sushi & Cocktails**\n• Адрес: Москва, Благовещенский переулок, 1А\n• Рейтинг: 5.0\n• Особенности:     │
│ Отличное место с атмосферным интерьером и вкусными роллами.\n\n- **Якитория**\n• Адрес: Москва, улица Маршала   │
│ Полубоярова, 1\n• Рейтинг: 5.0\n• Особенности: Хороший выбор для любителей горячих роллов и детских семейных    │
│ посещений.\n\nВыбирайте на свой вкус!'}                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: На основе отзывов рекомендуется посетить следующие японские рестораны в Москве:

- **Izumi**
• Адрес: Москва, Мясницкая улица, 38с1
• Рейтинг: 5.0
• Особенности: Одно из самых любимых мест с разнообразным меню и удобной системой вызова официантов.

- **Tsunami Sushi & Cocktails**
• Адрес: Москва, Благовещенский переулок, 1А
• Рейтинг: 5.0
• Особенности: Отличное место с атмосферным интерьером и вкусными роллами.

- **Якитория**
• Адрес: Москва, улица Маршала Полубоярова, 1
• Рейтинг: 5.0
• Особенности: Хороший выбор для любителей горячих роллов и детских семейных посещений.

Выбирайте на свой вкус!

[Step 2: Duration 42.88 seconds| Input tokens: 4,072 | Output tokens: 282]

In [ ]:
print(resp)

С учетом мнений посетителей, могу порекомендовать следующие японские рестораны в Москве:

- **Tsunami Sushi & Cocktails**
Адрес: Москва, Благовещенский переулок, 1А
Рейтинг: 5.0
Отзыв: Уютное место с неоновой атмосферой Токио и отличными роллами.
- **Izumi** (Мясницкая улица, 38с1)
Рейтинг: 5.0
Отзыв: Предлагает разнообразное меню при адекватной цене с акцентом на аутентичность японской кухни.
- **Якитория** (угол Маршала Полубоярова, д.1)
Рейтинг: 5.0
Отзыв: Рекомендуется для любителей роллов и горячих блюд с детским меню.
